In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from PIL import Image
import os

In [2]:
name=input("please enter the name of image with extension")
IMAGE_PATH = "Input_images"
IMAGE_PATH =os.path.join(IMAGE_PATH,name)

please enter the name of image with extensionSudoku.jpg


In [3]:
img=cv2.imread(IMAGE_PATH)
img=cv2.resize(img,(500,500))
image=img.copy()
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,51,9)
kernel = np.ones((5,5), np.uint8) 
thresh = cv2.dilate(thresh, kernel, iterations=1)
thresh = cv2.erode(thresh, kernel, iterations=1)
# cv2.imshow("x",thresh)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
img = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)


In [4]:
cnts = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
area_treshold = 1000
o=np.zeros((600,600))
# cv2.drawContours(img, cnts[0], -1, (0, 255, 0), 3)
center=[]
maxx=0
maxy=0
for c in cnts:
    x=cv2.minAreaRect(c)
    if(x[1][0]>maxx):
        maxx=x[1][0]
    if(x[1][1]>maxy):
        maxy=x[1][1]
     

for c in cnts:
   
#     cv2.drawContours(img, [c], -1, (0, 255, 0), 3)
#       x,y,w,h = cv2.boundingRect(c)
#       cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 3)
    x=cv2.minAreaRect(c)
    if x[1][0]==maxx:
        center.append(int(x[0][0]-x[1][1]/2))
        center.append(int(x[0][1]-x[1][0]/2))
print(f"{x[1][0]}    {x[1][1]}")
#     cv2.polylines(o,[b],True,(0,255,0),2)
#     break
# img=image[50:180, 100:300]
print(x)
# cv2.drawContours(img, cnts, -1, (0, 255, 0), 3)
# cv2.imshow("one",img)
# cv2.waitKey(0)

361.0    361.0
((249.5, 249.5), (361.0, 361.0), 90.0)


In [5]:
img=image[int(center[0]+5):int(center[0]+maxx-5),int(center[1]+5):int(center[1]+maxy-5)]

In [6]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
trans=transforms.Compose([
    transforms.Resize((50,50)),
    transforms.ToTensor(),
    transforms.Normalize([0.,0.,0.],
                        [1.0,1.0,1.0])
])

In [7]:
class one(nn.Module):
    def __init__(self):
        super().__init__()
        self.c1=nn.Conv2d(3,12,kernel_size=3,stride=1,padding=1)
        self.b1=nn.BatchNorm2d(12)
        self.r1=nn.ReLU()
        self.p1=nn.MaxPool2d(kernel_size = 2, stride=2)
        self.c2=nn.Conv2d(12,20,kernel_size=3,stride=1,padding=1)
        self.r2=nn.ReLU()
        self.c3=nn.Conv2d(20,32,kernel_size=3,stride=1,padding=1)
        self.r3=nn.ReLU()
        self.b3=nn.BatchNorm2d(32)
        self.l=nn.Linear(32*25*25,10)
    def forward(self,x):
        o=self.c1(x)
        o=self.b1(o)
        o=self.r1(o)
        o=self.p1(o)
        o=self.c2(o)
        o=self.r2(o)
        o=self.c3(o)
        o=self.b3(o)
        o=self.r3(o)
        o=o.view(-1,32*25*25)
        o=self.l(o)
        return o

In [8]:
checkpoint=torch.load('best.model',torch.device(device))
model=one()
model.load_state_dict(checkpoint)
model.eval()

one(
  (c1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (b1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (r1): ReLU()
  (p1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (c2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (r2): ReLU()
  (c3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (r3): ReLU()
  (b3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (l): Linear(in_features=20000, out_features=10, bias=True)
)

In [9]:
img=cv2.resize(img,(450,450))
a=[]
ans=[]
ans1=[]
k=0
for i in range(9):
    
    for j in range(9):
        d=int((j+1)/3)
        d1=int((i+1)/3)
        num=img[i*50+8+d1*2:(i+1)*50-10+d1*1,j*50+11+d*2:(j+1)*50-16+d*2]
        n=num.copy()
#         num=img[i*50+15:(i+1)*50-5,j*50+10:(j+1)*50-15]
        
        num=cv2.resize(num,(28,40))
        cv2.imwrite("store\\one.png",num)
        image=Image.open("store\\one.png")
        image_tensor=trans(image).float()
        image_tensor=image_tensor.unsqueeze_(0)
        
        input=torch.autograd.Variable(image_tensor)
        num=cv2.cvtColor(num,cv2.COLOR_BGR2GRAY)
        thresh = cv2.adaptiveThreshold(num,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,51,9)
        cnts,_ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
      
        if len(cnts)==0:
            print(0,end=" ")
            a.append(0)
            ans1.append([i,j])
        else:
            o=model(input)
            index=o.data.numpy().argmax()
            print(index,end=" ")
            a.append(index)
    ans.append(a)
    
    a=[]
    print()
# num=img[310:340,15:40]
# cv2.imshow("x",img)
# plt.imshow(num)
# plt.show()
# # cv2.imshow("y",num)
# cv2.imwrite("one.jpg",num)
# cv2.waitKey(0)

0 0 4 0 5 0 0 0 0 
9 0 0 7 3 4 6 0 0 
0 0 3 0 2 1 0 4 9 
0 3 5 0 9 0 4 8 0 
0 9 0 0 0 0 0 3 0 
0 7 6 0 1 0 9 2 0 
3 1 0 9 7 0 2 0 0 
0 0 9 1 8 2 0 0 3 
0 0 0 0 6 0 1 0 0 


In [10]:
print(len(ans1))

45


In [11]:
N=9

In [12]:

 
def isSafe(grid, row, col, num):
 

    for x in range(9):
        if grid[row][x] == num:
            return False
 

    for x in range(9):
        if grid[x][col] == num:
            return False
 
  
    startRow = row - row % 3
    startCol = col - col % 3
    for i in range(3):
        for j in range(3):
            if grid[i + startRow][j + startCol] == num:
                return False
    return True
 

def solveSudoku(grid, row, col):
 
 
    if (row == N - 1 and col == N):
        return True
     
    
    if col == N:
        row += 1
        col = 0
 
    
    if grid[row][col] > 0:
        return solveSudoku(grid, row, col + 1)
    for num in range(1, N + 1, 1):
     
        
        if isSafe(grid, row, col, num):
         
            
            grid[row][col] = num
 
            
            if solveSudoku(grid, row, col + 1):
                return True
 
      
        grid[row][col] = 0
    return False

In [14]:
if(solveSudoku(ans, 0, 0)):
    print("Sudoku solved...")
 

True

In [17]:
for i in range(9):
    
    for j in range(9):
        d=int((j+1)/3)
        d1=int((i+1)/3)
        
        if [j,i] in ans1:
            x=(i*50+8+d1*2+(i+1)*50-10+d1*1)/2-15
            y=(j*50+11+d*2+(j+1)*50-16+d*2)/2+15
            cv2.putText(img,f"{ans[j][i]}",(int(x),int(y)),cv2.FONT_HERSHEY_PLAIN,2,(0,0,255),2)
           
cv2.imshow("one",img)
OUTPUT_PATH=os.path.join("Results","Solved_"+name)
cv2.imwrite(OUTPUT_PATH,img)
cv2.waitKey(0)

-1